In [3]:
!mkdir save_frames

In [3]:
!mkdir predicted_frames

In [6]:
# importing the necessary libraries
import cv2
import numpy as np
import uuid
import os

# Creating a VideoCapture object to read the video
cap = cv2.VideoCapture('input/test_video.mp4')

count=0
while (cap.isOpened()):

    ret, frame = cap.read()
    
    filename=str(uuid.uuid4())+".jpg"
    fullpath=os.path.join("./save_frames",filename)
    cv2.imwrite(fullpath, frame)
    count+=1
    
    if count==20:
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
!ls

data			   models	    predict.py	 train.py
export_inference_graph.py  pipeline.config  save_frames  Untitled.ipynb
fine_tuned_model	   predicted.jpg    Thresh.jpg


In [22]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [1]:
predicted_path = './predicted_frames'

In [23]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import numpy as np
import uuid
import os
import time

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


KPIs_dict=dict()

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = "input/frozen_inference_graph.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = "input/labels.txt"

NUM_CLASSES =7



# ## Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


# ## Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

    
# ## Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def calculate_area(x1,y1,x2,y2):
    xDiff = abs(x1 - x2) 
    yDiff = abs(y1 - y2)
    area = xDiff * yDiff
    return area



# Size, in inches, of the output images.
IMAGE_SIZE = (500, 500)
count=0
frame_number=0
cap = cv2.VideoCapture('input/test_video.mp4')

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while (cap.isOpened()):
            frame_number+=1
            
            ret, frame = cap.read()
                        
            filename=str(uuid.uuid4())+".jpg"
            fullpath=os.path.join("./save_frames",filename)
            cv2.imwrite(fullpath, frame)
            count+=1

            ### for testing script...
            if count==50:
                break
            ### ends here
            
            image = Image.open(fullpath)

            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
              # Visualization of the results of a detection.

            image,box_to_display_str_map=vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)

            image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
            im_width, im_height = image_pil.size
            area_whole = im_width * im_height
            for key,value in box_to_display_str_map.items():
                ymin, xmin, ymax, xmax = key
                (left, right, top, bottom) = (xmin * im_width, xmax * im_width,ymin * im_height, ymax * im_height)
                area= calculate_area(top,left,bottom,right)
                
                percent_area = round(area/area_whole,2)
                rindex=value[0].rfind(':')
                brand_name = value[0][:rindex]
                
                if brand_name in KPIs_dict.keys():
                    
                    KPIs_dict[brand_name]['count']+=1
                    KPIs_dict[brand_name]['area'].append(percent_area)
                    KPIs_dict[brand_name]['frames'].append(frame_number)
                else:
                    KPIs_dict[brand_name]={"count":1}
                    KPIs_dict[brand_name].update({"area":[percent_area]})
                    KPIs_dict[brand_name].update({"frames":[frame_number]})


            full_predicted_path = os.path.join(predicted_path,str(uuid.uuid4())+".jpg")
            cv2.imwrite(full_predicted_path,image)
            

In [5]:
KPIs_dict

{'hotstar': {'count': 8,
  'area': [0.16, 0.16, 0.22, 0.18, 0.15, 0.34, 0.31, 0.16],
  'frames': [4, 5, 5, 6, 10, 14, 16, 34]},
 'pepsi': {'count': 2, 'area': [0.0, 0.0], 'frames': [46, 46]}}

In [19]:
for each_brand,analytics_dict in KPIs_dict.items():
    area=analytics_dict['area']
    response = shortest_longest_area(area)
    KPIs_dict[each_brand].update(response)

In [20]:
def shortest_longest_area(area_list):
    area_list.sort()
    shortest = area_list[0]
    longest = area_list[-1]
    response = {
        "shortest":shortest,
        "longest":longest
    }
    return response

In [21]:
KPIs_dict

{'hotstar': {'count': 8,
  'area': [0.15, 0.16, 0.16, 0.16, 0.18, 0.22, 0.31, 0.34],
  'frames': [4, 5, 5, 6, 10, 14, 16, 34],
  'shortest': 0.15,
  'longest': 0.34},
 'pepsi': {'count': 2,
  'area': [0.0, 0.0],
  'frames': [46, 46],
  'shortest': 0.0,
  'longest': 0.0}}